In [19]:
import torch
import torch.nn as nn
from torch.nn import init
import random
import numpy as np
'''
this model implement simplest cgan, that is:
concatnate latent and label together
and send it into generator
but generator and discriminator use pggan
but I failedXDD
i dont what's wrong in my codeQQ

'''

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU State:', device)


# Random seed
manualSeed = 7777
print('Random Seed:', manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

z_dim = 100
n_classes = 24
img_size = 64
batch_size = 128
#about pggan https://samaelchen.github.io/pytorch-pggan/
#and https://towardsdatascience.com/progressively-growing-gans-9cb795caebee

def linear_init(linear):
    #https://medium.com/@shoray.goel/kaiming-he-initialization-a8d9ed0b5899
    #detail of xavier and kaiming
    #https://chih-sheng-huang821.medium.com/%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92-weight-initialization%E5%92%8Cbatch-normalization-f264c4be37f5
    init.xavier_normal(linear.weight)
    linear.bias.data.zero_()
def conv_init(conv):
    init.kaiming_normal(conv.weight)
    if conv.bias is not None:
        conv.bias.data.zero_()

def upsampling2d(nhwf):
    _, h, w, _ = int_shape(nhwf)
    return tf.image.resize_nearest_neighbor(nhwf, (2 * h, 2 * w))
def downsampling2d(nhwf):
    return tf.nn.avg_pool(nhwf, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

#作者用的第一個trick
#讓每個weight的更新速度一樣
class EqualLR:
    def __init__(self, name):
        self.name = name
    def compute_weight(self, module):
        weight = getattr(module, self.name + '_orig')
        fan_in = weight.data.size(1) * weight.data[0][0].numel()
        return weight * np.sqrt(2 / fan_in)

    @staticmethod
    def apply(module, name):
        fn = EqualLR(name)
        weight = getattr(module, name)
        del module._parameters[name]
        # add new paras named NAME_org
        module.register_parameter(name + '_orig', nn.Parameter(weight.data))
        #new new paras without change the archi of model
        module.register_forward_pre_hook(fn)
        return fn
    def __call__(self, module, input):
        weight = self.compute_weight(module)
        setattr(module, self.name, weight)
        
def equal_lr(module, name='weight'):
    EqualLR.apply(module, name)
    return module
    
class EqualConv2d(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()

        conv = nn.Conv2d(*args, **kwargs)
        conv.weight.data.normal_()
        conv.bias.data.zero_()
        self.conv = equal_lr(conv)

    def forward(self, input):
        return self.conv(input)
class PixelNorm(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, input):
        return input / torch.sqrt(torch.mean(input ** 2, dim=1, keepdim=True)+ 1e-8)

       
class ConvBlock(nn.Module):
    #a block in generator
    def __init__(self, in_channel, out_channel, kernel1, pad1, kernel2, pad2, pixel_norm=True):
        super().__init__()

        self.kernel1 = kernel1
        self.kernel2 = kernel2
        self.stride1 = 1
        self.stride2 = 1
        self.pad1 = pad1
        self.pad2 = pad2

        if pixel_norm:
            self.conv = nn.Sequential(EqualConv2d(in_channel, out_channel, self.kernel1, self.stride1, self.pad1),
                                      PixelNorm(),
                                      #no bn or in layer, we use pixelnorm
                                      nn.LeakyReLU(0.2),
                                      
                                      EqualConv2d(out_channel, out_channel, self.kernel2, self.stride2, self.pad2),
                                      PixelNorm(),
                                      nn.LeakyReLU(0.2))
        else:
            self.conv = nn.Sequential(EqualConv2d(in_channel, out_channel, self.kernel1, self.stride1, self.pad1),
                                      nn.LeakyReLU(0.2),
                                      EqualConv2d(out_channel, out_channel, self.kernel2, self.stride2, self.pad2),
                                      nn.LeakyReLU(0.2))
            
    
    def forward(self, input):
        out = self.conv(input)

class EqualConv2d(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()

        conv = nn.Conv2d(*args, **kwargs)
        conv.weight.data.normal_()
        #動態init weight to balance lr
        #每過一次rgb就要用一次
        conv.bias.data.zero_()
        self.conv = equal_lr(conv)

    def forward(self, input):
        return self.conv(input)

class generator(nn.Module):
    
    def __init__(self, n_classes, img_size, z_dim):
        super(generator, self).__init__()
        
        self.n_classes = n_classes
        self.z_dim = z_dim
        self.in_size = n_classes + z_dim
        self.img_size = img_size
        
        self.label_emb = nn.Embedding(self.n_classes, self.n_classes)
        
        self.code_norm = PixelNorm()
        self.label_emb.weight.data.normal_()
        #in_channel, out_channel, kernel1, pad1, kernel2, pad2
        self.progression = nn.ModuleList([ConvBlock(self.in_size, self.in_size, 4, 3, 3, 1),
                                          ConvBlock(self.in_size, self.in_size, 3, 1, 3, 1),
                                          ConvBlock(self.in_size, self.in_size, 3, 1, 3, 1),
                                          ConvBlock(self.in_size, self.in_size, 3, 1, 3, 1),
                                          ConvBlock(self.in_size, int(self.in_size/2), 3, 1, 3, 1),
                                          ConvBlock(int(self.in_size/2), int(self.in_size/4), 3, 1, 3, 1)])

        self.to_rgb = nn.ModuleList([nn.Conv2d(self.in_size, 3, 1),
                                     nn.Conv2d(self.in_size, 3, 1),
                                     nn.Conv2d(self.in_size, 3, 1),
                                     nn.Conv2d(self.in_size, 3, 1),
                                     nn.Conv2d(int(self.in_size/2), 3, 1),
                                     nn.Conv2d(int(self.in_size/4), 3, 1)])
        
    def forward(self, noise, labels, expand=0, alpha=0):
        #noise should be a [bs, z_dim] dim tensor! RANDOM CHOOSE!
        #https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/cgan/cgan.py
        #gen_input is a [bs, z_dim+n_classes] tensor!
        
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)#(bs, feature_dim(z_dim + n_classes))
        gen_input = gen_input.view(gen_input.size(0), gen_input.size(-1), 1, 1)#(bs, feat_dim, 1, 1)
        out = self.code_norm(gen_input)
        
        for i, (conv, to_rgb) in enumerate(zip(self.progression, self.to_rgb)):
            if i > 0 and expand > 0:
                upsample = F.interpolate(out, scale_factor=2)
                out = conv(upsample)
            else:
                print(out)
                out = conv(out)
                
            if i == expand:#will run this block initially
                out = to_rgb(out)
                if i > 0 and 0 <= alpha < 1:
                    skip_rgb = self.to_rgb[i - 1](upsample)
                    out = (1 - alpha) * skip_rgb + alpha * out
                break
            
        return out
    
class discriminator(nn.Module):
    def __init__(self, n_classes):
        super(discriminator, self).__init__()
        
        self.n_classes = n_classes
        self.label_embedding = nn.Embedding(self.n_classes, self.n_classes)

        #also should be designed by myself
#         self.model = nn.Sequential(
#             nn.Linear(opt.n_classes + int(np.prod(img_shape)), 512),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Linear(512, 512),
#             nn.Dropout(0.4),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Linear(512, 512),
#             nn.Dropout(0.4),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Linear(512, 1),
#         )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity

GPU State: cpu
Random Seed: 7777
